# Exercise 4 

We assume that we have a pipe of infinitisimal thickness. We can parametrize the surface of the pipe with two dimensional polorid coordinates and we will arrive at the approximated two dimensional heat equation:

$$
\left\{ \begin{array}{rll}
\partial_t u(t,\theta,z ) &= \frac{1}{r^2} \partial^2_{\theta} u(t,\theta, z ) + \partial_{z}^2 u(t,\theta, z ) \qquad & \text{for} \; (\theta, z ) \in \Omega := [0, 2 \pi ) \times [ 0, 1 ] \\
u(0,\cdot,\cdot) &= u_0 \\
u(t,\cdot,z) &= 0 \qquad & \text{for} \; z \in \{ 0, 1 \}
\end{array}
\right.
$$

where $ \frac{1}{3} =: r > 0$ denotes the radius of the pipe. Note that $u$ is periodic in $\theta$.

For $u_0$ we choose:
$$
u_0(\theta, z ) := 
\left\{ 
\begin{array}{lcl} 
	1 & \qquad & \text{if} \; 0 \le \theta \le \frac{\pi}{8} \; \text{and} \; \left|z - \frac{1}{2} \right| \le 10^{-1}\\
	0 & \qquad &\text{else}\\
	\end{array}
	\right.
$$

### a )

Create the discretized values $L_h, u_{h,0}$ as before. For the grid use $n = (150,100)$. Solve the resulting linear ODE with the help of the matrix exponential as in No.2 on sheet 4. Plot the result for $t=0$, $t=10^{-2}$ and $t = 5 \cdot 10^{-2}$ side by side in a surface plot.

In [4]:
import numpy as np
from scipy.sparse import diags, eye, kron, csc_matrix
from scipy.sparse.linalg import expm_multiply
import matplotlib.pyplot as plt
%matplotlib qt
import math

def get_bvp(n1,n2):
    h1 = 2*np.pi/(n1+1) #step size for theta grid
    h2 = 1/(n2+1)       #step size for z grid
    
    #Construct T matrix
    d1 = -( (2*9)/(h1**2) + 2/(h2**2) )*np.ones(n1)  #main diagonal
    d2 =  9/(h1**2)*np.ones(n1-1)  #upper and lower diagonal
    T = diags([d1,d2,d2,9/(h1**2),9/(h1**2)],[0,-1,1,-n1+1,n1-1])

    #Construct I matrix
    In2 = diags(np.ones(n2),0) 
    
    #Construct Lh
    T = kron(In2,T) #diagonal T
    
    I = (1/(h2**2))*(eye(n1*n2,n1*n2,-n1)+eye(n1*n2,n1*n2,n1))  #I_N  (this is for z grid part) 
    Lh = -(T + I)
    
    #Construct initial u0(theta,z)
    u0 = np.zeros((n1*n2,1))
    for i in range(n1):
        for j in range(n2):
            if (i+1)*h1 <= np.pi/8 and abs( (j+1)*h2 - 0.5 ) <= 0.1:
                u0[j*n1+i] = 1
    
    return Lh,u0

n1 = 100
n2 = 100
Lh,u0 = get_bvp(n1,n2)

#Plot surface
tlist = [0,0.1,0.05] #t=0,0.1,0.5
fig = plt.figure(figsize=plt.figaspect(0.5))
for k in tlist:
    Lht = -Lh * k  #L_h * t 
    uh = expm_multiply(Lht,u0) # uh = exp(L_h t)u_0(x_i)   
    theta = np.linspace(0,2*np.pi,n1+2)[1:-1]
    z = np.linspace(0,1,n2+2)[1:-1]
    Uh = np.zeros((n1,n2)) #Construct matrix u_h
    for i in range(n1):
        for j in range(n2):
            Uh[i][j] = uh[j*n1+i]
    
    z, theta = np.meshgrid(z, theta)
    ax = fig.add_subplot(1, 3, tlist.index(k)+1, projection='3d')
    ax.plot_surface(z, theta, Uh,cmap='viridis', edgecolor='none')
    ax.set_title('Surface plot when t={}'.format(k))
    ax.set_xlabel('z')
    ax.set_ylabel('theta')
    ax.set_zlabel('u_h')
    plt.show()
    
# Please wait for around 16 seconds

### b)

Compute the quantity 

$$
	H_h(t) := \frac{2 \pi }{ n_1 n_2  } \sum_{i,j} u_h(t,\theta_i, z_j)^2
$$

for $t_0 = 0$ and $t_1,\dots, t_{9}$ evenly logarithmitcly spaced between $10^{-5}$ and $10^{-2}$. Plot the result with linear scaled axes.

In [14]:

def H(t):
    Lht = -Lh * t  #-L_h * t 
    uh = expm_multiply(Lht,u0) #solution uh
    H = 2 * np.pi * sum([i**2 for i in uh])/n1/n2 # get H
    return float(H)
    
#Your code
t = np.logspace(-5,-2,9) 
t = np.insert(t,0,0)  #time space

for l in t:
    plt.scatter(l,H(l),c='brown') #plot t vs H
plt.title('Plot of H(t)')
plt.xlabel('t')
plt.ylabel('H')

Text(0, 0.5, 'H')